In [15]:
import time
from pyspark.sql import SQLContext
sql = SQLContext(sc)

In [16]:
wikipediadata = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="projet", table="wikipediadata")

In [17]:
wikipediadata.printSchema()

root
 |-- projectcode: string (nullable = true)
 |-- page: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- views: long (nullable = true)



In [20]:
day_min = 60
day_max = 61

In [21]:
sums = wikipediadata.where((wikipediadata.day >= day_min) & (wikipediadata.day <= day_max)).\
    groupby('projectcode', 'page', 'day').sum('views')

In [22]:
sums.cache()

DataFrame[projectcode: string, page: string, day: int, SUM(views): bigint]

In [23]:
t0 = time.time()
sums.take(10)
t1 = time.time()

In [24]:
d = t1-t0
m = int(d // 60)
s = int(d % 60)
print('La requête a pris {} min {} secondes'.format(m, s))

La requête a pris 4 min 7 secondes


In [26]:
sums.take(2)

[Row(projectcode=u'nl', page=u'Rembrandtplein_(Amsterdam)', day=61, SUM(views)=6),
 Row(projectcode=u'de', page=u'Bladeserver', day=60, SUM(views)=100)]

In [25]:
4*60+7

247